# Uniswap Helper Functions (Calculations)

In [1]:
import math
q96 = 2 ** 96

## Square root price x96

In [2]:
def price_to_sqrtp(price_of_token0, price_of_token1, decimals_of_token0, decimals_of_token1):
    adj_p = (price_of_token1 / price_of_token0) * (10 ** decimals_of_token1 / 10 ** decimals_of_token0)
    return int(math.sqrt(adj_p) * q96)

price_to_sqrtp(2000,1,6,18)

1771595571142957166518320255467520

In [4]:
def sqrtPrice_to_price(sqrtPriceX96, decimals_of_token0, decimals_of_token1):
  return ((sqrtPriceX96/q96) ** 2) * (10 ** decimals_of_token0 / 10 ** decimals_of_token1)

p = sqrtPrice_to_price(1771595571142957166518320255467520, 6, 18)
print(p)

0.0005
